In [1]:
# !pip install opencv-contrib-python --user

In [2]:
import numpy as np
import cv2
import os
import pickle
import sys
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

In [3]:
# Load the trained KMeans model 
with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

In [4]:
n_classes=36
clustering_factor=6


def orb_features(images):
  orb_vectors_list={}
  orb_descriptors_list=[]
  orb=cv2.ORB_create()
  for key,value in images.items():
    print(key, "Started")
    features=[]
    for img in value:
      kp,desc=orb.detectAndCompute(img,None)
      orb_descriptors_list.extend(desc)
      features.append(desc)
    orb_vectors_list[key]=features
    print(key," Completed!")
  return [orb_descriptors_list,orb_vectors_list]

In [5]:
# Loading train images into dictionaries which holds all images category by category

def load_images_by_category(folder):
  images={}
  for label in os.listdir(folder):
    print(label," started")
    category=[]
    path=folder+'/'+label
    for image in os.listdir(path):
      img=cv2.imread(path+'/'+image)
      #new_img=cv2.resize(img,(128,128))
      if img is not None:
        category.append(img)
    images[label]=category
    print(label, "ended")
  return images

In [6]:
# Creating histograms for train images

# Function takes 2 parameters. The first one is a dictionary that holds the descriptors that are separated class by class 
# And the second parameter is thekm model
# Returns a dictionary that holds the histograms for each images that are separated class by class. 

def create_histogram(all_bows,kmeans):
  features_dict={}
  for key,value in all_bows.items():
    print(key," Started!")
    category=[]
    for desc in value:
      visual_words=kmeans.predict(desc)
      hist = np.array(np.bincount(visual_words,minlength=n_classes*clustering_factor))
      category.append(hist)
    features_dict[key]=category
    print(key," Completed!")
  return features_dict

# all_test_descriptors = orb_test[0]
# kmeans = minibatchkmeans(n_classes*clustering_factor,all_test_descriptors)


In [7]:
test_folder='C:/Users/Sandeep Roy/Desktop/data2/data/Train-Test/Test'

# Loading Test images
test_images=load_images_by_category(test_folder)

#Extract ORB features from the image
orb_test=orb_features(test_images)[1]

#print(len(orb_test['a'][0]))

# Create histograms from extracted sift features
bows_test=create_histogram(orb_test,kmeans)

In [ ]:
import csv
loc='C:/Users/Sandeep Roy/Desktop/data2/data/test.csv'
with open(loc,'w',newline='') as file:
  writer=csv.writer(file)
  header=[]
  for i in range (1,n_classes*clustering_factor+1):
    header.append(str('pixel')+str(i))
  header.append('Label')
  writer.writerow(header)
  count=0
  for label in bows_test:
    #print(len(bows_test[label]))
    for i in range(len(bows_test[label])):
      list=[]
      for j in range(150):
        list.append(bows_test[label][i][j])
      list.append(label)
      writer.writerow(list)